# Import packages

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings 
warnings.filterwarnings("ignore")

# Import data

In [ ]:
df = pd.read_csv('../data/base_modelis_pass_det.csv',sep = ';')

# Dataviz rapide

In [ ]:
df.isna().sum()

In [ ]:
df.head(20)

# Data treatment

In [ ]:
df['date_entree_defaut'] = pd.to_datetime(df['date_entree_defaut'])

In [ ]:
df['arrete'] = pd.to_datetime(df['arrete'])

In [ ]:
df_validation = df[df['arrete']>"2020-12-30"]

On split les données en un ensemble de train et un de test, toutes les données anterieures 2021 seront dans le train et le reste dans le test

In [ ]:
data = df[df['arrete']<"2020-12-30"]

On va garder une seule ligne par identifiant client, la plus récente pour chaque client, pour cela on doit trier par la date au sein de chaque groupe 'cd_op'

In [ ]:
data = data.groupby('cd_op').apply(lambda x: x.sort_values(by='arrete', ascending=False))

Ensuite on ne va garder que la dernière occurence de chaque cd_op pour le logit

In [ ]:
data_logit = data.drop_duplicates(subset = 'cd_op', keep = 'first')

#### On va récupérer les colonnes contenant bcp de NaN dans le jeu de test pour les retirer aussi du train_test

In [ ]:
df_validation = df_validation.groupby('cd_op').apply(lambda x: x.sort_values(by='arrete', ascending=False))

In [ ]:
condition = df_validation.drop_duplicates(subset='cd_op', keep='first').isna().sum()/df_validation.shape[0] < 0.2 # on supprime les colonnes qui contiennent + de 20% de NaN
df_filtered = df_validation.loc[:, condition]

# Correlation
### Variables numeriques

In [ ]:
(np.abs(data[df_filtered.columns].drop_duplicates(subset='cd_op', keep='first').select_dtypes(include = 'number').corr()['fl_pass_DET'])>0.1)==True

### Variables categorielles

In [ ]:
for col in data[df_filtered.columns].drop_duplicates(subset='cd_op', keep='first').select_dtypes(include = 'object').nunique()

In [ ]:
from scipy.stats import chi2_contingency

liste_var_cat_dependantes = []

for col in data[df_filtered.columns].drop_duplicates(subset='cd_op', keep='first').select_dtypes(include = 'object').columns:
    contingency_table = pd.crosstab(data['fl_pass_DET'], data[col])
    
    chi2, p, _, _ = chi2_contingency(contingency_table)
    
    if p<0.05:
        liste_var_cat_dependantes.append(col)
        print(f"p-value de {col}:", p)
    else:
        pass
    


# 01-Logit simple

In [ ]:
data_logit['fl_pass_DET'].value_counts()

In [ ]:
# Sélectionner les données
X = data[df_filtered.columns] # On drop les var qui ont trop de NaN
X.drop(columns = ['cum_dec','cum_rec','MR_arr'],inplace = True)
X = X.select_dtypes(include = 'number')
X.dropna(inplace = True)

y = X['fl_pass_DET']

X.drop(columns = 'fl_pass_DET', inplace = True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_prob)

y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Obtenir les coefficients du modèle
coefficients = model.coef_[0]

# Obtenir les noms des caractéristiques
feature_names = X.columns

# Créer un DataFrame avec les noms des caractéristiques et leurs coefficients
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Trier le DataFrame par coefficient en valeur absolue pour avoir une visualisation plus claire
feature_importance_df['Abs_Coefficient'] = np.abs(feature_importance_df['Coefficient'])
feature_importance_df = feature_importance_df.sort_values(by='Abs_Coefficient', ascending=False)

# Tracer le graphique des importances des caractéristiques
plt.figure(figsize=(12, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Abs_Coefficient'], color='skyblue')
plt.xlabel('Coefficient de régression (en valeur absolue)')
plt.ylabel('Caractéristique')
plt.title('Importance des caractéristiques dans le modèle de régression logistique')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
# Faire des prédictions de probabilités
y_prob = model.predict_proba(X_test)[:, 1]

# Calculer la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Calculer l'AUC
roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()

# Test pour voir la variable parasite

In [ ]:
# Sélectionner les données
X = data[['MT_FMG','max_age_ctx','cum_frs_externes','dls_mep_entree_def','NB_DUR_PFI','ANC_PRO_MAX_PFI','cum_dec','cum_rec','AGE_EMP1','MT_EAD_RESID','MR_arr','dls_ges_def','MT_EAD_DNE_CHR_d','MT_CHA_HORS_OPE_PFI','CD_POST_BIEN_PFI','EAD_1an_avant_def','CD_ETAT_CIVIL','CD_DNE_CHR_LGD_SAIN','fl_pass_DET']] # On drop les var qui ont trop de NaN


X.dropna(inplace = True)

y = X['fl_pass_DET']

X.drop(columns = 'fl_pass_DET', inplace = True)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Supposons que X sont vos caractéristiques et y est votre variable cible
# Assurez-vous que X et y sont des DataFrames ou des tableaux NumPy

# Créer une liste pour stocker les résultats
results = []

# Boucle sur chaque variable
for variable in X.columns:
    # Sélectionner la variable courante
    X_variable = X[[variable]]
    
    # Diviser les données
    X_train, X_test, y_train, y_test = train_test_split(X_variable, y, test_size=0.2, random_state=42)
    
    # Créer et entraîner le modèle de régression logistique
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Faire des prédictions de probabilités
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Calculer l'AUC
    auc = roc_auc_score(y_test, y_prob)
    
    # Ajouter les résultats à la liste
    results.append({'Variable': variable, 'AUC': auc})

# Créer un DataFrame à partir de la liste de résultats
auc_df = pd.DataFrame(results)

# Afficher les résultats
print(auc_df)
